In [3]:
import random
import math
import matplotlib.pyplot as plt
import networkx as nx
import experiments_func as ef
import block_division_func as bd

In [4]:
import random
import networkx as nx
import math

# Define the problem parameters
num_computers = 10
num_tasks = 5000
num_blocks_total = 15000

G = nx.read_edgelist("/home/yamamoto/research/consideration_of_computer_power/src/data/japanese_network.edgelist", data=False)

# トポロジにある計算機の数
num_nodes = G.number_of_nodes()
# トポロジにある計算機にキャパシティを割り当てる
capacities = ef.generate_random_numbers(num_nodes, num_blocks_total)

# ランダムにノードを {num_pcs}個取得
random_nodes = ef.get_random_nodes(graph=G, num_pcs=num_computers)
# ランダムノードのキャパシティ
random_capacities = [capacities[node] for node in random_nodes]

random_assigned_matrix = bd.generate_random_assignment(num_pcs=num_computers, num_blocks=num_tasks, capacities=random_capacities)

initial_state = random_assigned_matrix
# for i in range(num_tasks):
#     initial_state.append(random.randint(0, num_computers - 1))
# initial_state = [1, 0, 2, 2, 3, 2, 0, 0, 1, 0, 3, 3]

# ニューラルネットワークの分割ブロックの構造を表す
structure_row = 2 # ブロックの行数
structure_col = num_tasks // structure_row # ブロックの列数
# ニューラルネットワークを分割したブロック
block_structure = bd.generate_block_structure(row=structure_row, col=structure_col)
# 通信する必要のあるブロックのリストを作成(ブロックの接続関係)
connection_relations = bd.generate_linked_block_list(block_structure)

# G = nx.Graph()
# G.add_nodes_from(range(num_computers))
# for i in range(num_tasks):
#     for j in connection_relations[i]:
#         G.add_edge(initial_state[i], initial_state[j])

# connection_relations = [[1, 2, 3], [0, 2, 3], [0, 1, 3, 4, 5], [0, 1, 2, 4, 5], [5], [4]]

# Create a graph to compute communication costs

# Function to compute the communication cost
def compute_communication_cost(state, converter):
    total_cost = 0
    for i in range(num_tasks):
        for j in connection_relations[i]:
            cost = nx.dijkstra_path_length(G, str(converter[state[i]]), str(converter[state[j]]))
            total_cost += cost
    return total_cost

# Function to generate a neighboring state
def generate_neighbor(state):
    neighbor = state.copy()
    task1, task2 = random.sample(range(num_tasks), 2)
    neighbor[task1], neighbor[task2] = neighbor[task2], neighbor[task1]
    return neighbor

# Function to calculate the acceptance probability
def acceptance_probability(delta_cost, temperature):
    if delta_cost < 0:
        return 1.0
    return math.exp(-delta_cost / temperature)

# Simulated annealing algorithm
def simulated_annealing(initial_state, converter, max_iterations, initial_temperature, cooling_rate):
    current_state = initial_state.copy()
    best_state = current_state.copy()
    current_cost = compute_communication_cost(current_state, converter)
    best_cost = current_cost
    temperature = initial_temperature

    for iteration in range(max_iterations):
        neighbor = generate_neighbor(current_state)
        neighbor_cost = compute_communication_cost(neighbor, converter)
        delta_cost = neighbor_cost - current_cost

        if acceptance_probability(delta_cost, temperature) > random.random():
            current_state = neighbor
            current_cost = neighbor_cost

        if current_cost < best_cost:
            best_state = current_state.copy()
            best_cost = current_cost

        temperature *= cooling_rate

        if iteration % 1000 == 0:
            print(f'iteration: {iteration+1000}')
            print(f'current_state: {current_state}')
            print(f'current_cost: {current_cost}\n')
    return best_state, best_cost

# Set the annealing parameters
max_iterations = 10000
initial_temperature = 100.0
cooling_rate = 0.95

# Solve the problem using simulated annealing
best_solution, best_cost = simulated_annealing(initial_state, random_nodes, max_iterations, initial_temperature, cooling_rate)
best_cost = compute_communication_cost(best_solution, random_nodes)

# Print the best solution and its cost
print("Best_solution:", best_solution)
print("Communication_cost:", best_cost)


iteration: 1000
current_state: [2, 4, 3, 7, 0, 4, 3, 7, 3, 9, 2, 6, 3, 3, 2, 2, 8, 6, 7, 7, 3, 2, 7, 9, 7, 9, 6, 4, 4, 2, 7, 2, 8, 8, 4, 4, 0, 4, 5, 8, 3, 0, 3, 2, 4, 4, 0, 6, 9, 0, 7, 0, 7, 3, 7, 2, 0, 6, 0, 7, 0, 7, 9, 5, 9, 2, 9, 5, 0, 0, 8, 0, 9, 8, 9, 7, 4, 1, 9, 2, 3, 2, 7, 6, 8, 9, 6, 1, 6, 9, 7, 0, 8, 8, 7, 6, 7, 9, 0, 9, 8, 6, 0, 2, 9, 6, 4, 7, 7, 6, 2, 2, 2, 9, 9, 4, 6, 9, 4, 9, 6, 0, 6, 4, 3, 7, 6, 7, 7, 7, 6, 0, 4, 4, 2, 0, 4, 3, 3, 4, 6, 0, 3, 3, 4, 0, 7, 9, 4, 9, 7, 6, 8, 8, 5, 8, 6, 7, 4, 7, 7, 7, 4, 3, 7, 2, 2, 0, 3, 6, 0, 6, 6, 9, 7, 0, 9, 7, 9, 0, 2, 2, 5, 0, 2, 3, 6, 6, 0, 7, 6, 9, 6, 0, 4, 6, 9, 0, 5, 0, 5, 7, 0, 6, 2, 9, 3, 7, 6, 3, 8, 6, 9, 9, 9, 8, 0, 7, 0, 8, 4, 3, 3, 9, 4, 0, 4, 2, 0, 6, 6, 3, 8, 7, 9, 2, 7, 2, 6, 7, 2, 0, 5, 2, 8, 6, 6, 5, 6, 4, 2, 8, 2, 4, 7, 7, 6, 7, 7, 0, 7, 7, 8, 0, 2, 6, 9, 9, 2, 6, 8, 8, 0, 2, 4, 7, 5, 7, 6, 4, 6, 3, 7, 9, 6, 4, 4, 9, 0, 2, 4, 7, 7, 6, 4, 2, 0, 7, 2, 2, 7, 6, 7, 7, 7, 9, 4, 4, 0, 7, 7, 3, 6, 0, 3, 7, 5, 2, 7, 8, 0, 8, 3,

In [5]:
import random
import networkx as nx
from deap import base, creator, tools

# Problem-specific constants
NUM_TASKS = 12
NUM_COMPUTERS = 5
CAPACITY_FACTOR = 1.5
INITIAL_STATE = [1, 0, 2, 2, 3, 2, 0, 0, 1, 4, 3, 3]
CONNECTION_RELATIONS = [[1, 2, 3], [0, 2, 3], [0, 1, 3, 4, 5], [0, 1, 2, 4, 5], [5], [4]]

G = nx.read_edgelist("/home/yamamoto/research/consideration_of_computer_power/src/data/japanese_network.edgelist", data=False)

# Create the fitness and individual classes
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Initialize the DEAP toolbox
toolbox = base.Toolbox()

# Define the individual initialization function
def create_individual():
    return [random.randint(0, NUM_COMPUTERS-1) for _ in range(NUM_TASKS)]

# Define the fitness evaluation function
def evaluate_fitness(individual):
    total_cost = 0
    G = nx.Graph()
    G.add_nodes_from(range(NUM_COMPUTERS))
    for i in range(NUM_TASKS):
        for j in CONNECTION_RELATIONS[i]:
            cost = nx.dijkstra_path_length(G, str(individual[i]), str(individual[j]))
            total_cost += cost
    return total_cost,

# Register the required DEAP functions
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_fitness)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=NUM_COMPUTERS-1, indpb=0.1)

def main():
    random.seed(42)

    # Create an initial population
    population = toolbox.population(n=100)

    # Evaluate the initial population
    fitnesses = map(toolbox.evaluate, population)
    for individual, fitness in zip(population, fitnesses):
        individual.fitness.values = fitness

    # Begin the evolution
    for generation in range(50):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Apply crossover and mutation on the offspring
        offspring = [toolbox.clone(ind) for ind in offspring]
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.8:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_individuals = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_individuals)
        for ind, fitness in zip(invalid_individuals, fitnesses):
            ind.fitness.values = fitness

        # Replace the population with the offspring
        population[:] = offspring

        # Print the best fitness in each generation
        best_individual = tools.selBest(population, k=1)[0]
        print(f"Generation {generation+1}: Best Fitness = {best_individual.fitness.values[0]}")

    print("Evolution completed.")

if __name__ == "__main__":
    main()


NodeNotFound: Node 0 not found in graph